In [1]:
import pickle
import pandas as pd

In [2]:
with open(f'../data/all_mf_am_230430.pkl', 'rb') as file:
    mf_am_data_raw = pickle.load(file)

In [3]:
symbol = 'MMM'
symbol_data_raw = mf_am_data_raw[symbol]
display(symbol_data_raw.keys())

dict_keys(['timestamp', 'income_statement', 'key_metrics_ttm', 'exchange'])

In [4]:
# Acquirer's Multiple Calculations
key_metrics_ttm = symbol_data_raw['key_metrics_ttm']
income_statement = symbol_data_raw['income_statement']

enterprise_value = key_metrics_ttm['enterpriseValueTTM']
revenue = income_statement['revenue']
cost_of_revenue = income_statement['costOfRevenue']
gross_profit = income_statement['grossProfit']
operating_expenses = income_statement['operatingExpenses']
operating_income = income_statement['operatingIncome']
ebitda = income_statement['ebitda']
depreciation_and_amortization = income_statement['depreciationAndAmortization']
ebit = ebitda - depreciation_and_amortization
acquirers_multiple = enterprise_value / ebit

print(f"enterprise_value:         {enterprise_value:,}")
print(f"ebit:                     {ebit:,}")
print(f"acquirers_multiple:       {acquirers_multiple:.2f}")

# Magic Formula Calculations
market_cap = key_metrics_ttm['marketCapTTM']
roic =  key_metrics_ttm['roicTTM']
earnings_yield =  key_metrics_ttm['earningsYieldTTM']

print(f"market_cap:               {market_cap:,}")
print(f"roic:                     {roic:.2f}")
print(f"earnings_yield:           {earnings_yield:.2f}")

enterprise_value:         70,997,599,840
ebit:                     4,401,000,000
acquirers_multiple:       16.00
market_cap:               58,598,599,840.0
roic:                     0.16
earnings_yield:           0.09


In [5]:
df_columns = ['symbol', 'exchange', 'market_cap', 'roic', 'earnings_yield', 'enterprise_value', 'acquirers_multiple']
mf_am_df_raw = pd.DataFrame(columns=df_columns)

display(mf_am_df_raw)

,symbol,exchange,market_cap,roic,earnings_yield,enterprise_value,acquirers_multiple


In [9]:
key_metrics_type = 'key_metrics_ttm'

for key in mf_am_data_raw:
    # print(key)

    symbol_data = mf_am_data_raw[key]

    if not key_metrics_type in symbol_data:
        print(f'{key}: {key_metrics_type} not found')
        continue

    key_metrics = symbol_data[key_metrics_type]
    income_statement = symbol_data['income_statement']

    if not isinstance(key_metrics, pd.Series):
        print(f'{key}: {key_metrics_type} is None')
        continue

    if key_metrics.empty:
        print(f'{key}: {key_metrics_type} is Empty')
        continue

    if key_metrics_type == 'key_metrics_ttm':
        # year = 'ttm'
        market_cap = key_metrics['marketCapTTM']
        roic =  key_metrics['roicTTM']
        earnings_yield =  key_metrics['earningsYieldTTM']
    else:
        year = key_metrics.columns[0]
        market_cap = key_metrics[year]['marketCap']
        roic =  key_metrics[year]['roic']
        earnings_yield =  key_metrics[year]['earningsYield']

    if not market_cap or not roic or not earnings_yield:
        print(f'{key}: empty val found')
        continue

    enterprise_value = key_metrics['enterpriseValueTTM']

    if enterprise_value <= 0:
        print(f'{key}: enterprise value is 0')
        continue
    
    if 'operatingIncome' not in income_statement:
        print(f'{key}: operatingIncome not found')
        continue

    if not income_statement['operatingIncome']:
        print(f'{key}: operatingIncome not found')
        continue

    operating_income = income_statement['operatingIncome']

    if operating_income <= 0:
        print(f'{key}: operating income is 0')
        continue

    acquirers_multiple = enterprise_value / operating_income

    symbol_mf_dict = {
        'symbol': key,
        'exchange': symbol_data['exchange'],
        'market_cap': market_cap,
        'roic': roic,
        'earnings_yield': earnings_yield,
        'enterprise_value': enterprise_value,
        'acquirers_multiple': acquirers_multiple,
    }

    mf_df2 = pd.DataFrame(symbol_mf_dict, index=[0])
    mf_am_df_raw = pd.concat([mf_df2, mf_am_df_raw], ignore_index=True)

display(mf_am_df_raw.head())
display(len(mf_am_df_raw))

LEX.L: operating income is 0
ARR.TO: operating income is 0
F-PC: empty val found
IMNN: operating income is 0
MBSC-UN: operating income is 0
SKYX: operating income is 0
SSU: operating income is 0
MHN.L: operating income is 0
AUUDW: empty val found
SIBN: operating income is 0
PRCT: operating income is 0
ACA.PA: enterprise value is 0
SAS.ST: operating income is 0
ALLO: operating income is 0
ACT.L: enterprise value is 0
TAO.V: operating income is 0
OSUR: operating income is 0
BRYN.DE: empty val found
IFRX: operating income is 0
SKYAW: empty val found
ELUX-B.ST: operating income is 0
EOLU-B.ST: operating income is 0
SERT.ST: operating income is 0
VRAR: operating income is 0
MAFL.L: operating income is 0
RSKD: operating income is 0
NRGV: operating income is 0
BNNR: operating income is 0
5UR.DE: empty val found
LAHAV.TA: operating income is 0
FEIM: operating income is 0
IMRN: enterprise value is 0
III.TO: operating income is 0
ATO.PA: operating income is 0
DTSS: operating income is 0
NDM.TO: 

,symbol,exchange,market_cap,roic,earnings_yield,enterprise_value,acquirers_multiple
0,ORK.OL,OSE,7.638123e+10,0.073316,0.067814,93286230400,16.280319
1,INCR,NASDAQ,3.846530e+08,0.096063,0.033620,407223956,7.712283
2,IMMOU.BR,EURONEXT,1.793041e+07,0.052386,0.181368,32717411,14.102332
3,GWRS,NASDAQ,2.618649e+08,0.024703,0.021660,365697870,34.312054
4,GYM.L,LSE,1.851460e+08,0.128323,-0.001049,600145984,8.066478


13400

In [10]:
# Drop NA
mf_am_df = mf_am_df_raw.copy()
mf_am_df = mf_am_df.dropna()

# Drop negative AM
# mf_am_df = mf_am_df.drop(mf_am_df.index[mf_am_df['acquirers_multiple'] < 0])
# mf_am_df = mf_am_df[(mf_am_df['acquirers_multiple'] < 0)]

# Magic Formula Ranking
mf_am_df["roic_rank"] = mf_am_df["roic"].rank(method="min", ascending=False)
mf_am_df["ey_rank"] = mf_am_df["earnings_yield"].rank(method="min", ascending=False)
mf_am_df["mf_score"] = mf_am_df["roic_rank"] + mf_am_df['ey_rank']
mf_am_df["mf_rank"] = mf_am_df["mf_score"].rank(method="min", ascending=True)

# Acquirers Multiple Ranking
mf_am_df["am_rank"] = mf_am_df["acquirers_multiple"].rank(method="min", ascending=True)

# Combined Ranking
mf_am_df["mf_am_score"] = mf_am_df["mf_rank"] + mf_am_df['am_rank']
mf_am_df["mf_am_rank"] = mf_am_df["mf_am_score"].rank(method="min", ascending=True)

# Clean up unused columns
mf_am_df_cleaned = mf_am_df.drop(columns=['roic', 'earnings_yield', 'enterprise_value', 'roic_rank', 'ey_rank', 'mf_score', 'mf_am_score'])
# mf_am_df_cleaned = mf_am_df.drop(columns=['roic', 'earnings_yield', 'enterprise_value'])
# mf_am_df_cleaned = mf_am_df

mf_am_df_sorted = mf_am_df_cleaned.sort_values(by=['mf_am_rank']).reset_index(drop=True)
mf_am_df_sorted.index += 1
display(mf_am_df_sorted.head())
len(mf_am_df_sorted)

,symbol,exchange,market_cap,acquirers_multiple,mf_rank,am_rank,mf_am_rank
1,PNT,NASDAQ,8.179864e+08,0,1.0,1.0,1.0
2,NCAC,NASDAQ,9.819956e+07,0,5.0,1.0,2.0
3,PERE.L,LSE,1.881028e+06,0,19.0,1.0,3.0
4,MTEM,NASDAQ,2.214054e+07,0,21.0,1.0,4.0
5,XTP.F,XETRA,7.456625e+07,0,23.0,1.0,5.0


12956

In [11]:
with open(f'../data/all_mf_am_230430_processed.pkl', 'wb') as file:
    pickle.dump(mf_am_df_cleaned, file)